# Categorical Features in Regression Models

So far, we have fit linear and $k$-nearest neighbors regression models to data where all of the features are quantitative. But what if all or some of the features are categorical? In theory, the solution is simple: we simply transform the categorical variables into quantitative variables using dummy (i.e., one-hot) encoding. However, in practice, some care is needed to ensure that the categorical variables are transformed in a consistent way between the training and the test data.

In [27]:
import pandas as pd
import numpy as np
df_housing = pd.read_csv("AmesHousing.txt", sep="\t")
df_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


## One Categorical Feature

Let's develop some intuition about the predictions that a regression model will make when there is a single categorical feature. First, suppose we train a linear regression model to predict house price from the neighborhood the house is in.

In [28]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

X = df_housing[["Neighborhood"]] # need 2D array for sklearn
y = df_housing["SalePrice"]

enc = OneHotEncoder() # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
X_dummies = enc.fit_transform(X)



In [29]:
model = LinearRegression()
model.fit(X_dummies, y)

LinearRegression()

A regression model with just a single feature, **Neighborhood**, will predict the same price for all houses in the same neighborhood. What is that predicted value? We can obtain it by applying the `OneHotEncoder` to a list of the unique neighborhoods in the data set and passing this to `model.predict()`.

One way to obtain a list of the unique neighborhoods is inside the encoder itself, under the attribute `.categories_`. We convert this to a 2D-array to be compatible with scikit-learn.

In [30]:
X_test = pd.Series(enc.categories_[0], name="Neighborhood").to_frame()
X_test

,Neighborhood
0,Blmngtn
1,Blueste
2,BrDale
3,BrkSide
4,ClearCr
5,CollgCr
6,Crawfor
7,Edwards
8,Gilbert
9,Greens


In [31]:
model.predict(enc.transform(X_test))

array([196661.67853343, 143589.99996525, 105608.33329502, 124756.24993639,
       208662.09086777, 201803.43463885, 207550.83489043, 130843.38119445,
       190646.57563488, 193531.24996556, 279999.99996649, 103752.90316937,
       136999.99996665,  95756.48644677, 162226.63151193, 145097.34992143,
       140710.86952827, 188406.90831796, 330319.12671214, 322018.26493565,
       123991.89178118, 135071.93745786, 136751.15221795, 184070.1839257 ,
       229707.32400895, 324229.19603555, 246599.54161792, 248314.5832962 ])

It is a bit hard to tell which prediction corresponds to which neighborhood. Let's put these numbers into a `Series`, indexed by the neighborhood.

In [32]:
pd.Series(
    model.predict(enc.transform(X_test)),
    index=X_test["Neighborhood"]
)

Neighborhood
Blmngtn    196661.678533
Blueste    143589.999965
BrDale     105608.333295
BrkSide    124756.249936
ClearCr    208662.090868
CollgCr    201803.434639
Crawfor    207550.834890
Edwards    130843.381194
Gilbert    190646.575635
Greens     193531.249966
GrnHill    279999.999966
IDOTRR     103752.903169
Landmrk    136999.999967
MeadowV     95756.486447
Mitchel    162226.631512
NAmes      145097.349921
NPkVill    140710.869528
NWAmes     188406.908318
NoRidge    330319.126712
NridgHt    322018.264936
OldTown    123991.891781
SWISU      135071.937458
Sawyer     136751.152218
SawyerW    184070.183926
Somerst    229707.324009
StoneBr    324229.196036
Timber     246599.541618
Veenker    248314.583296
dtype: float64

Could we have obtained these predictions some other way, without going through the trouble of fitting a linear regression model? Intuitively, if all we knew about a house was the neighborhood it was in, we would predict the average price of houses in that neighborhood.

In [33]:
df_housing.groupby("Neighborhood")["SalePrice"].mean()

Neighborhood
Blmngtn    196661.678571
Blueste    143590.000000
BrDale     105608.333333
BrkSide    124756.250000
ClearCr    208662.090909
CollgCr    201803.434457
Crawfor    207550.834951
Edwards    130843.381443
Gilbert    190646.575758
Greens     193531.250000
GrnHill    280000.000000
IDOTRR     103752.903226
Landmrk    137000.000000
MeadowV     95756.486486
Mitchel    162226.631579
NAmes      145097.349887
NPkVill    140710.869565
NWAmes     188406.908397
NoRidge    330319.126761
NridgHt    322018.265060
OldTown    123991.891213
SWISU      135071.937500
Sawyer     136751.152318
SawyerW    184070.184000
Somerst    229707.324176
StoneBr    324229.196078
Timber     246599.541667
Veenker    248314.583333
Name: SalePrice, dtype: float64

These numbers match the predictions from our linear regression model exactly. Linear regression simply predicts the average price in each neighborhood. 

To see this mathematically, recall that linear regression minimizes the total squared distance between the observed price and the predicted price:

$$ \text{sum of } (\text{price} - \widehat{\text{price}})^2. $$

After we expand the **Neighborhood** column into 28 dummy variables (e.g., $I\{ \text{Blmngtn} \}$, $I\{ \text{Blueste} \}$, etc.), one for each neighborhood, we can write the predicted price in the linear regression model as 

$$ \widehat{\text{price}} = c_1 I\{ \text{Blmngtn} \} + c_2 I\{ \text{Blueste} \} + \ldots + c_{28} I\{ \text{Veenker} \}. $$

(For simplicity, we have omitted the intercept term $b$.)

Now, consider a house in Bloomington Heights, for which $I\{ \text{Blmngtn} \} = 1$ and all of the other dummy variables $I\{ \text{Blueste} \} = \ldots = I\{ \text{Veenker} \} = 0$. Then, $\widehat{\text{price}}$ for a house in Bloomington Heights is $c_1$. Likewise, $\widehat{\text{price}}$ for a house in Bluestem is $c_2$. And so forth.

Now, we can reframe linear regression as learning the values $c_1, c_2, \ldots, c_{28}$ that minimize

$$ \text{sum of } (\text{price} - \widehat{\text{price}})^2 = \underbrace{\text{sum of } (\text{price} - c_1)^2}_{\text{over houses in Blmngtn}} + \underbrace{\text{sum of } (\text{price} - c_2)^2}_{\text{over houses in Blueste}} + \ldots + \underbrace{\text{sum of } (\text{price} - c_{28})^2}_{\text{over houses in Veenker}}. $$ 

The value of $c$ that mimimizes the $\text{sum of } (\text{price} - c)^2$ is the mean of the prices. So $\hat c_1$ will be the average price of houses in Bloomington Heights, $\hat c_2$ the average price of houses in Bluestem, and so on. Since $\hat c_1, \hat c_2, \ldots, \hat c_{28}$ are also the predicted values for each neighborhood, this shows that linear regression will predict the average label in each category when there is only one categorical variable in the model.

Exercise 1 in this lesson asks you to investigate what $k$-nearest neighbors regression does in the same situation.

## Mixing Quantitative and Categorical Features

In general, we want to fit machine learning models that use a mix of both categorical and quantitative features. In this situation, we will want to apply the `OneHotEncoder` to only the categorical features. Scikit-learn provides a `ColumnTransformer` that allows us to selectively apply transformations to certain columns.

For example, suppose we want to fit a $k$-nearest neighbors model to predict house price from quantitative features (square footage, number of bedrooms, number of full bathrooms) and categorical features (neighborhood, building type). We can use a `ColumnTransformer` to standardize the quantitative features and one-hot encode the categorical features.

In [34]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (StandardScaler(), ["Gr Liv Area", "Bedroom AbvGr", "Full Bath"]),
    (OneHotEncoder(), ["Neighborhood", "Bldg Type"]),
    remainder="drop"  # all other columns in X will be dropped.
)
ct

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['Gr Liv Area', 'Bedroom AbvGr', 'Full Bath']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Neighborhood', 'Bldg Type'])])

Next, we integrate this `ColumnTransformer` into a pipeline (refer to the previous lesson) with the `KNeighborsRegressor` model.

In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=10)
)

pipeline.fit(X=df_housing[["Gr Liv Area", "Bedroom AbvGr", "Full Bath",
                           "Neighborhood", "Bldg Type"]], 
             y=df_housing["SalePrice"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'Bedroom AbvGr',
                                                   'Full Bath']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Neighborhood',
                                                   'Bldg Type'])])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=10))])

Now, if we wanted to use this model to predict the price of a 3BR/2BA, 1700 sqft single-family house in Bloomington Heights, we could create a `Series` with this information, and call `pipeline.predict()` on a 2D-array with this single row.

In [36]:
x_test = pd.Series()
x_test["Gr Liv Area"] = 1700
x_test["Bedroom AbvGr"] = 3
x_test["Full Bath"] = 2
x_test["Neighborhood"] = "Blmngtn"
x_test["Bldg Type"] = "1Fam"

pipeline.predict(X=pd.DataFrame([x_test]))

/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_92268/248949205.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x_test = pd.Series()


array([251550.])

So this house is predicted to cost $251,550.

## Exercises

1\. Using the Ames data set, build a $10$-nearest neighbors model to predict house price using **Neighborhood** as the only feature. How do the predictions compare with just using the mean house price of each neighborhood? If there are any discrepancies, can you explain why?

In [37]:
ames_df = pd.read_csv("AmesHousing.txt", sep='\t', index_col="Order")
#ames_df.head()
ames_train = ames_df.loc[:2344].copy()
ames_test = ames_df.loc[2345:].copy()

# Log transform the target.
ames_train["log(SalePrice)"] = np.log(ames_train["SalePrice"])
ames_train.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,log(SalePrice)
Order,,,,,,,,,,,,,,,,,,,,,
1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2010,WD,Normal,215000,12.278393
2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000,11.561716
3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000,12.055250
4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2010,WD,Normal,244000,12.404924
5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900,12.154253


In [38]:
X_train = ames_train[["Neighborhood"]]
y_train = ames_train["SalePrice"]

home = pd.DataFrame({
    'Neighborhood' : [3]
})

# Fit k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=10)
model.fit(
    X=ames_train[["Neighborhood"]],
    y=ames_train["SalePrice"]
)

# Make predictions at those feature values.
model.predict(
    X = ames_test[["Neighborhood"]]
)

array([193780. , 193780. , 161400. , 193780. , 281066.8, 193780. ,
       215220. , 193780. , 161400. , 193780. , 193780. , 193780. ,
       281066.8, 193780. , 193780. , 193780. , 193780. , 193780. ,
       193780. , 161400. , 193780. , 193780. , 161400. , 193780. ,
       193780. , 193780. , 161400. , 193780. , 161400. , 161400. ,
       193780. , 193780. , 161400. , 193780. , 193780. , 281066.8,
       281066.8, 161400. , 161400. , 281066.8, 161400. , 193780. ,
       281066.8, 281066.8, 281066.8, 193780. , 193780. , 281066.8,
       193780. , 281066.8, 281066.8, 193780. , 193780. , 161400. ,
       161400. , 161400. , 281066.8, 193780. , 193780. , 161400. ,
       161400. , 161400. , 161400. , 191919.6, 161400. , 161400. ,
       193780. , 193780. , 193780. , 193780. , 281066.8, 193780. ,
       193780. , 281066.8, 193780. , 161400. , 161400. , 161400. ,
       161400. , 161400. , 161400. , 161400. , 161400. , 161400. ,
       191919.6, 161400. , 281066.8, 193780. , 193780. , 19378

In [39]:
model.predict(home)

array([193780.])

2\. In the example from the lesson, we standardized the quantitative features and one-hot encoded the categorical features in parallel. This means that the dummy variables were not standardized before being passed into the $10$-nearest neighbors model. How would you modify the pipeline so that *all* of the variables are standardized?

(_Hint:_ You may find the `remainder="passthrough"` option of `ColumnTransformer` helpful.)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (StandardScaler(), ["Gr Liv Area", "Bedroom AbvGr", "Full Bath"]),
    (OneHotEncoder(), ["Neighborhood", "Bldg Type"]),
    remainder="passthrough"
)
ct

3\. Using the tips data set (tips.csv ), use a $5$-nearest neighbors model to predict how much a male diner will tip on a Sunday bill of \$40.00.

In [40]:
tips_df = pd.read_csv("tips.csv")

tips_df['day'] = tips_df['day'].replace("Sun" , '7').replace("Sat" , '1').replace("Fri" , '5').replace("Thu" , '4').replace("Wed" , '3').replace("Tue" , '2').replace("Mon" , '1').astype(float)

tips_df['sex'] = tips_df['sex'].replace("M" , '1').replace("F" , '2').astype(float)

tips_train = tips_df.loc[:200].copy()
tips_test = tips_df.loc[201:].copy()

# Log transform the target.
#tips_train["tip"] = np.log(tips_train["tip"])
tips_train.head()

,obs,totbill,tip,sex,smoker,day,time,size
0,1,16.99,1.01,2.0,No,7.0,Night,2
1,2,10.34,1.66,1.0,No,7.0,Night,3
2,3,21.01,3.50,1.0,No,7.0,Night,3
3,4,23.68,3.31,1.0,No,7.0,Night,2
4,5,24.59,3.61,2.0,No,7.0,Night,4


In [41]:
X_train = tips_train[["totbill", "day", "sex"]]
y_train = tips_train["tip"]

z = pd.DataFrame({
    'totbill' : [40],
    'day' : [7],
    'sex' : [1],
})

# Fit k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=5)
model.fit(
    X=tips_train[["totbill", "day", "sex"]],
    y=tips_train["tip"]
)

# Make predictions at those feature values.
model.predict(
    X = tips_test[["totbill", "day", "sex"]]
)

array([2.214, 2.15 , 3.5  , 3.204, 3.5  , 3.506, 5.096, 3.574, 2.394,
       3.28 , 3.506, 5.846, 2.6  , 3.298, 2.394, 3.298, 1.792, 1.428,
       3.28 , 1.962, 2.19 , 1.534, 2.726, 1.9  , 2.726, 1.966, 3.156,
       2.194, 3.532, 4.134, 2.226, 1.792, 1.72 , 2.226, 1.658, 1.946,
       5.214, 5.49 , 3.28 , 3.786, 3.762, 3.082, 2.86 ])

In [42]:
model.predict(z)

array([4.346])